### CS102/CS103

Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
NUI Galway

# Lecture 16: Matrices

In Mathematics, matrices are data objects of fundamental importance.
How can we deal with matrices in `python`?

## Implementing Matrices

One possible way to implement a matrix using known `python` data structures is a `list` of `list`s: A matrix is a list of
(row) vectors, and a vector is a list of numbers.
Of course, all the rows of a matrix have the same length ...

In [1]:
A = [
[1,2,-1,-1,0],
[7,5,-6,1,8],
[3,3,3,-4,1],
[6,-5,3,0,4]]
A

[[1, 2, -1, -1, 0], [7, 5, -6, 1, 8], [3, 3, 3, -4, 1], [6, -5, 3, 0, 4]]

For starters, it would be nice to see a matrix printed in a more "matrix-like" fashion ...

In [2]:
def str_matrix(m):
    text = ["[\n"]
    for row in m:
        text.append(" {}\n".format(row))
    text.append("]")
    return "".join(text)

In [3]:
print(str_matrix(A))

[
 [1, 2, -1, -1, 0]
 [7, 5, -6, 1, 8]
 [3, 3, 3, -4, 1]
 [6, -5, 3, 0, 4]
]


##  Row Reduction

**Gaussian elimination** is an algorithm that transforms 
any given matrix $A$ into an equivalent matrix in  **row echolon form**
by a systematic
application of **elementary row operations**.
The algorithm can be used to find the **inverse**
of a (square, invertible) matrix, and to solve
systems of linear equations, like
$$
\begin{array}{rcl}
x + 2y - x - w &=& 0\\
7x + 5y - 6z + w &=& 8\\
3x + 3y + 3z - 4w &=& 1\\
6x - 5y + 3z &=& 4
\end{array}
$$

The **algorithm** can be described as follows.

**Input:** an $m \times n$-matrix $A = (a_{ij})$

**Output:** an equivalent matrix in row reduced form.

2. Choose a **pivot**, i.e., an entry $a = a_{ij} \neq 0$ such that
all columns to the left of $a_{ij}$ consist entirely of zeros.

1. **Stop** if not pivot can be found (then $A = 0$ and an all-zero matrix is in row reduced form.)

3. Otherwise, if $i > 1$, **swap rows** $1$ and $i$ (so that the
pivot lies in the top row.

4. **Clear the column** below the pivot: for each $i$ ($i > 1$),
replace row $i$ by row $i$ minus $c$ times row $1$,
where $c = -a_{ij}^{-1}/a$, to produce an entry $0$ under the
pivot in row $i$.

5. Enter **recursion**: Let $B$ be the matrix obtained from $A$ by ignoring the first row and go back to step 1 with $B$ in place of $A$.

Let's try and implement this in `python`.  It will be convenient
to apply the row operations **in place** (thereby destroying the original matrix ...).
Recall that list positions in `python` start at $0$
So the **first** row of a matrix represented as list `A`
will be `A[0]`.

We apply a divide-and-conquer strategy (a top-down design)
and implement the algorithm in terms of subroutines,
leaving it to the (yet to be written) functions that implement
those subroutines to worry about the details.

In [4]:
def echelon_form(A, row, col):
    "transform a matrix A into row echelon form"
    
    # 1. find a pivot
    pivot = pivot_coordinates(A, row, col)
    
    # 2. return if matrix is a zero matrix
    if pivot == None: 
        return
    
    # 3. make pivot row the 'first' row
    i, j = pivot
    if i > row:
        row_op_swap(A, i, row)
    
    # 4. clear column under pivot
    clear_column(A, row, j)
    
    # 5. enter recursion
    echelon_form(A, row + 1, col + 1)

Unfortunately, we can't test this function before all the gaps
are closed.  

Let's implement elementary **row operations** next.  In place, i.e.,
rather than returning a value, these functions will **modify**
the entries of a given matrix ...

In [5]:
def row_op_swap(A, i, k):
    "swap rows i and k"
    A[i], A[k] = A[k], A[i]

In [6]:
def row_op_scale(A, i, c):
    "scale row i by c != 0"
    for j in range(len(A[i])):
        A[i][j] = c * A[i][j]

In [7]:
def row_op_add(A, i, k, c):
    "add c-multiple of row k to row i"
    for j in range(len(A[i])):
        A[i][j] = A[i][j] + c * A[k][j]

That's one gap filled (`row_op_swap()`).
Finding a **pivot** is straight-forward if you do it by hand.
How do you instruct a computer program to do this?

In [8]:
def pivot_coordinates(A, i, j):
    "find first nonzero entry in matrix A, or None"
    m, n = len(A), len(A[1])
    while j < n:
        while i < m and A[i][j] == 0:
            i += 1
        if i < m:
            return i, j
        j += 1

This function we can test:

In [9]:
pivot_coordinates(A, 0, 0)

(0, 0)

The next gap to fill is `clear_column()`.  An opportunity
to apply `row_op_add()` ...

In [10]:
def clear_column(A, row, col):
    for i in range(row + 1, len(A)):
        row_op_add(A, i, row, -A[i][col]/A[row][col])

And that's it:

In [11]:
echelon_form(A, 0, 0)

In [12]:
print(str_matrix(A))

[
 [1, 2, -1, -1, 0]
 [0.0, -9.0, 1.0, 8.0, 8.0]
 [0.0, 0.0, 5.666666666666667, -3.6666666666666665, -1.6666666666666665]
 [0.0, 0.0, 8.881784197001252e-16, -4.509803921568628, -9.019607843137255]
]


## Back Substitution

The Jordan part of the **Gauss-Jordan** procedure
continues with further row operations to produce the
**reduced** row echelon form (aka row **canonical_form**) of
the matrix:

For each nonzero row $R_k$:

1. if the pivot $a$ in row $R_k$ is not $1$, replace
$R_k$ by $\frac1a R_k$ to **make the pivot equal to $1$**.

2. **clear the column** above the pivot: to each row $i$
add $c R_k$, where $c = -a_{ik}$, to produce an entry $0$
above the pivot.

Use a version of `echelon_form()` that `return`s the positions of
the pivots in the resulting matrix (they'll be in rows $1$ to $k$,
so only the column coordinates are needed ...)

In [13]:
def echelon_form(A, row, col):
    pivot = pivot_coordinates(A, row, col)
    
    # 1. return if matrix is a zero matrix
    if pivot == None: 
        return []
    
    # 2. make pivot row the 'first' row
    i, j = pivot
    if i > row:
        row_op_swap(A, i, row)
    
    # clear column under pivot
    clear_column(A, row, j)
    
    # enter recursion
    pivots = echelon_form(A, row + 1, col + 1)
    
    return [j] + pivots

In [14]:
def canonical_form(A):
    pivots = echelon_form(A, 0, 0)
    for i in reversed(range(len(pivots))):
        clear_column_above(A, i, pivots[i])

`clear_column_above()` then uses the row operations
`row_op_scale()` and `row_op_add()`.

In [15]:
def clear_column_above(A, row, col):
    if A[row][col] != 1:
        row_op_scale(A, row, 1/A[row][col])
    for k in range(row):
        row_op_add(A, k, row, -A[k][col])        

That's it. Let's find the row canonical form of the matrix $A$.

In [16]:
A = [
[1,2,-1,-1,0],
[7,5,-6,1,8],
[3,3,3,-4,1],
[6,-5,3,0,4]]
canonical_form(A)
A

[[1.0, 0.0, 0.0, 0.0, 1.0],
 [-0.0, 1.0, -0.0, -0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [-0.0, -0.0, -1.9694391045524513e-16, 1.0, 1.9999999999999998]]

Hmm. Rounding errors ... Let's print the entries in text fields
of width 8, with only 1 digit after the decimal dot.

In [23]:
print("'{:8.1f}'".format(A[1][4]))
print("'12345678'")

'     1.0'
'12345678'


In [24]:
def str_matrix(m):
    text = ["[\n"]
    for row in m:
        entries = []
        for entry in row:
            entries.append("{:8.1f}".format(entry))        
        text.append(" [{}],\n".format(", ".join(entries)))
    text.append("]")
    return "".join(text)

In [25]:
print(str_matrix(A))

[
 [     1.0,      0.0,      0.0,      0.0,      1.0],
 [    -0.0,      1.0,     -0.0,     -0.0,      1.0],
 [     0.0,      0.0,      1.0,      0.0,      1.0],
 [    -0.0,     -0.0,     -0.0,      1.0,      2.0],
]


So $x = y = z = 1$ and $w = 2$.

Finally, if we add reporting functionality to the row operations,
we can watch Gaussian elimination in action ...

In [28]:
def row_op_swap(A, i, k):
    "swap rows i and k"
    print("A[{}] <-> A[{}]".format(i, k))
    A[i], A[k] = A[k], A[i]

In [35]:
def row_op_scale(A, i, c):
    "scale row i by c != 0"
    print("A[{0}] <- {1:.1f} * A[{0}]".format(i, c))
    for j in range(len(A[i])):
        A[i][j] = c * A[i][j]

In [36]:
def row_op_add(A, i, k, c):
    "add c-multiple of row k to row i"
    print("A[{0}] <- A[{0}] + {2:.1f} * A[{1}]".format(i, k, c))
    for j in range(len(A[i])):
        A[i][j] = A[i][j] + c * A[k][j]

In [37]:
A = [
[1,2,-1,-1,0],
[7,5,-6,1,8],
[3,3,3,-4,1],
[6,-5,3,0,4]]
canonical_form(A)
A

A[1] <- A[1] + -7.0 * A[0]
A[2] <- A[2] + -3.0 * A[0]
A[3] <- A[3] + -6.0 * A[0]
A[2] <- A[2] + -0.3 * A[1]
A[3] <- A[3] + -1.9 * A[1]
A[3] <- A[3] + -1.3 * A[2]
A[3] <- -0.2 * A[3]
A[0] <- A[0] + 1.0 * A[3]
A[1] <- A[1] + -8.0 * A[3]
A[2] <- A[2] + 3.7 * A[3]
A[2] <- 0.2 * A[2]
A[0] <- A[0] + 1.0 * A[2]
A[1] <- A[1] + -1.0 * A[2]
A[1] <- -0.1 * A[1]
A[0] <- A[0] + -2.0 * A[1]


[[1.0, 0.0, 0.0, 0.0, 1.0],
 [-0.0, 1.0, -0.0, -0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [-0.0, -0.0, -1.9694391045524513e-16, 1.0, 1.9999999999999998]]